In [0]:
!!git clone https://github.com/deepanrajm/deep_learning.git
from keras.preprocessing.image import img_to_array
from keras.applications import ResNet50
from keras.applications import imagenet_utils
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import imutils
from google.colab.patches import cv2_imshow
import time
import cv2

In [0]:
# initialize variables used for the object detection procedure
INPUT_SIZE = (350, 350)
PYR_SCALE = 1.5
WIN_STEP = 16
ROI_SIZE = (224, 224)
BATCH_SIZE = 64


In [0]:
def sliding_window(image, step, ws):
	# slide a window across the image
	for y in range(0, image.shape[0] - ws[1], step):
		for x in range(0, image.shape[1] - ws[0], step):
			# yield the current window
			yield (x, y, image[y:y + ws[1], x:x + ws[0]])

def image_pyramid(image, scale=1.5, minSize=(224, 224)):
	# yield the original image
	yield image

	# keep looping over the image pyramid
	while True:
		# compute the dimensions of the next image in the pyramid
		w = int(image.shape[1] / scale)
		image = imutils.resize(image, width=w)

		# if the resized image does not meet the supplied minimum
		# size, then stop constructing the pyramid
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
			break

		# yield the next image in the pyramid
		yield image

def classify_batch(model, batchROIs, batchLocs, labels, minProb=0.5,
	top=10, dims=(224, 224)):
	# pass our batch ROIs through our network and decode the
	# predictions
	preds = model.predict(batchROIs)
	P = imagenet_utils.decode_predictions(preds, top=top)

	# loop over the decoded predictions
	for i in range(0, len(P)):
		for (_, label, prob) in P[i]:
			# filter out weak detections by ensuring the
			# predicted probability is greater than the minimum
			# probability
			if prob > minProb:
				# grab the coordinates of the sliding window for
				# the prediction and construct the bounding box
				(pX, pY) = batchLocs[i]
				box = (pX, pY, pX + dims[0], pY + dims[1])

				# grab the list of predictions for the label and
				# add the bounding box + probability to the list
				L = labels.get(label, [])
				L.append((box, prob))
				labels[label] = L

	# return the labels dictionary
	return labels

In [0]:
# load our the network weights from disk
print("[INFO] loading network...")
model = ResNet50(weights="imagenet", include_top=True)

# initialize the object detection dictionary which maps class labels
# to their predicted bounding boxes and associated probability
labels = {}

In [0]:
# load the input image from disk and grab its dimensions
orig = cv2.imread("deep_learning/Object-Detection/beagle.png")
(h, w) = orig.shape[:2]

In [0]:
# resize the input image to be a square
resized = cv2.resize(orig, INPUT_SIZE, interpolation=cv2.INTER_CUBIC)

# initialize the batch ROIs and (x, y)-coordinates
batchROIs = None
batchLocs = []

In [0]:
# start the timer
print("[INFO] detecting objects...")
start = time.time()

In [0]:
# start the timer
print("[INFO] detecting objects...")
start = time.time()

In [0]:
# loop over the image pyramid
for image in image_pyramid(resized, scale=PYR_SCALE,
	minSize=ROI_SIZE):
	# loop over the sliding window locations
	for (x, y, roi) in sliding_window(resized, WIN_STEP, ROI_SIZE):
		# take the ROI and pre-process it so we can later classify the
		# region with Keras
		roi = img_to_array(roi)
		roi = np.expand_dims(roi, axis=0)
		roi = imagenet_utils.preprocess_input(roi)

		# if the batch is None, initialize it
		if batchROIs is None:
			batchROIs = roi

		# otherwise, add the ROI to the bottom of the batch
		else:
			batchROIs = np.vstack([batchROIs, roi])

		# add the (x, y)-coordinates of the sliding window to the batch
		batchLocs.append((x, y))

		# check to see if our batch is full
		if len(batchROIs) == BATCH_SIZE:
			# classify the batch, then reset the batch ROIs and
			# (x, y)-coordinates
			labels = classify_batch(model, batchROIs, batchLocs,
				labels)

			# reset the batch ROIs and (x, y)-coordinates
			batchROIs = None
			batchLocs = []


In [0]:
# check to see if there are any remaining ROIs that still need to be
# classified
if batchROIs is not None:
	labels = classify_batch(model, batchROIs, batchLocs, labels)

In [0]:
# show how long the detection process took
end = time.time()
print("[INFO] detections took {:.4f} seconds".format(end - start))

In [0]:
# loop over the labels for each of detected objects in the image
for k in labels.keys():
  # clone the input image so we can draw on it
	clone = resized.copy()

	# loop over all bounding boxes for the label and draw them on
	# the image
	for (box, prob) in labels[k]:
		(xA, yA, xB, yB) = box
		cv2.rectangle(clone, (xA, yA), (xB, yB), (0, 255, 0), 2)

	# show the image *without* apply non-maxima suppression
	cv2_imshow(clone)
	clone = resized.copy()

	# grab the bounding boxes and associated probabilities for each
	# detection, then apply non-maxima suppression to suppress
	# weaker, overlapping detections
	boxes = np.array([p[0] for p in labels[k]])
	proba = np.array([p[1] for p in labels[k]])
	boxes = non_max_suppression(boxes, proba)

	# loop over the bounding boxes again, this time only drawing the
	# ones that were *not* suppressed
	for (xA, yA, xB, yB) in boxes:
		cv2.rectangle(clone, (xA, yA), (xB, yB), (0, 0, 255), 2)

	# show the output image - with NMS
	print("[INFO] {}: {}".format(k, len(boxes)))
	cv2_imshow(clone)
	cv2.waitKey(0)